In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

In [3]:
from summer2 import CompartmentalModel, Stratification, Multiply
from summer2.parameters import Parameter
from summer2.functions.time import get_piecewise_function as pcwise_fcn
from summer2.functions.time import get_linear_interpolation_function as linear_interp_fcn
from summer2.functions.time import get_sigmoidal_interpolation_function as sigmoidal_interp_fcn

**IQR**

In [4]:
m = CompartmentalModel(
times=[0, 268],
compartments=["I", "Q", "R"],
infectious_compartments=["I"],
timestep = 1,
ref_date=datetime(2020,4,7)
)
m.set_initial_population({"I": 100000.0})
m.add_transition_flow("notification", Parameter("detection_rate"),"I","Q")
m.add_transition_flow("recovery_from_Q", Parameter("recovery_rate_Q"),"Q","R")
m.add_transition_flow("recovery_from_I", Parameter("recovery_rate_I"),"I","R")
#Requesting Outputs
m.request_output_for_flow("recovery_from_Q", "recovery_from_Q")
m.request_output_for_flow("recovery_from_I", "recovery_from_I")
m.request_aggregate_output("recoveries", ["recovery_from_Q", "recovery_from_I"], True)

DerivedOutput recoveries {'request_type': 'agg', 'sources': ['recovery_from_Q', 'recovery_from_I'], 'save_results': True}

In [8]:
param_set = {"recovery_rate_I": 0.1, "recovery_rate_Q": 1/7.5, "detection_rate": 0.4}
m.run(param_set)

In [9]:
m.get_outputs_df().plot()

/Users/mark/miniconda3/envs/summer2/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [10]:
m.get_derived_outputs_df().plot()

/Users/mark/miniconda3/envs/summer2/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



**I_kQ_kR**

In [13]:
m2 = CompartmentalModel(
times=[0, 268],
compartments=["I1", "I2", "I3", "I4", "Q1", "Q2", "Q3", "Q4", "R"],
infectious_compartments=["I1", "I2", "I3", "I4"],
timestep = 1,
ref_date=datetime(2020,4,7)
)
m2.set_initial_population({"I1": 100000.0})
m2.add_transition_flow("detection_I1_to_Q1", Parameter("detection_rate"),"I1","Q1")
m2.add_transition_flow("detection_I2_to_Q2", Parameter("detection_rate"),"I2","Q2")
m2.add_transition_flow("detection_I3_to_Q3", Parameter("detection_rate"),"I3","Q3")
m2.add_transition_flow("detection_I4_to_Q4", Parameter("detection_rate"),"I4","Q4")
m2.add_transition_flow("recovery_I1_to_I2", 4*Parameter("recovery_rate_I"),"I1","I2")
m2.add_transition_flow("recovery_I2_to_I3", 4*Parameter("recovery_rate_I"),"I2","I3")
m2.add_transition_flow("recovery_I3_to_I4", 4*Parameter("recovery_rate_I"),"I3","I4")
m2.add_transition_flow("recovery_I4_to_R", 4*Parameter("recovery_rate_I"),"I4","R")
m2.add_transition_flow("recovery_Q1_to_Q2", 4*Parameter("recovery_rate_Q"),"Q1","Q2")
m2.add_transition_flow("recovery_Q2_to_Q3", 4*Parameter("recovery_rate_Q"),"Q2","Q3")
m2.add_transition_flow("recovery_Q3_to_Q4", 4*Parameter("recovery_rate_Q"),"Q3","Q4")
m2.add_transition_flow("recovery_Q4_to_R", 4*Parameter("recovery_rate_Q"),"Q4","R")
#Requesting Outputs
m2.request_output_for_flow("recovery_I4_to_R", "recovery_I4_to_R")
m2.request_output_for_flow("recovery_Q4_to_R", "recovery_Q4_to_R")
m2.request_aggregate_output("recoveries", ["recovery_I4_to_R", "recovery_Q4_to_R"], True)

DerivedOutput recoveries {'request_type': 'agg', 'sources': ['recovery_I4_to_R', 'recovery_Q4_to_R'], 'save_results': True}

In [14]:
m2.run(param_set)

In [15]:
m2.get_outputs_df().plot()

/Users/mark/miniconda3/envs/summer2/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [16]:
m2.get_derived_outputs_df().plot()

/Users/mark/miniconda3/envs/summer2/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



**Comparing the two**

In [21]:
epoch1 = m.get_epoch()
epoch2 = m2.get_epoch()

fig = go.Figure()
fig = fig.add_trace(go.Scatter(x = epoch1.index_to_dti(m.times), y = m.get_derived_outputs_df()['recovery_from_I'], name = "IQR_recovery_ItoR"))
fig = fig.add_trace(go.Scatter(x = epoch2.index_to_dti(m2.times), y = m2.get_derived_outputs_df()['recovery_I4_to_R'], name = "I_kQ_kR_recovery_I4toR"))
fig = fig.add_trace(go.Scatter(x = epoch1.index_to_dti(m.times), y = m.get_derived_outputs_df()['recovery_from_Q'], name = "IQR_recovery_QtoR"))
fig = fig.add_trace(go.Scatter(x = epoch2.index_to_dti(m2.times), y = m2.get_derived_outputs_df()['recovery_Q4_to_R'], name = "I_kQ_kR_recovery_Q4toR"))
fig = fig.add_trace(go.Scatter(x = epoch1.index_to_dti(m.times), y = m.get_derived_outputs_df()['recoveries'], name = "IQR_recovery"))
fig = fig.add_trace(go.Scatter(x = epoch2.index_to_dti(m2.times), y = m2.get_derived_outputs_df()['recoveries'], name = "I_kQ_kR_recovery"))
fig.show()